## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [ ]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

In [ ]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [ ]:
from sqlalchemy import select
from sqlalchemy.orm import Session

# Usando o modelo ORM mapeado automaticamente para a tabela tracks
Tracks = Base.classes.tracks

session = Session(engine)

stmt = select(Tracks).limit(3)
result = session.scalars(stmt)

print("=== Três primeiros registros da tabela tracks ===")
for track in result:
    print(track.TrackId, "-", track.Name)

In [ ]:
from sqlalchemy import select
from sqlalchemy.orm import Session

Tracks = Base.classes.tracks
Albums = Base.classes.albums

session = Session(engine)

stmt = (
    select(Tracks.Name, Albums.Title)
    .join_from(Tracks, Albums, Tracks.AlbumId == Albums.AlbumId)
    .limit(10)
)

result = session.execute(stmt)

print("=== Nome da faixa e título do álbum das 10 primeiras faixas ===")
for track_name, album_title in result:
    print(f"Faixa: {track_name:40s} | Álbum: {album_title}")

In [ ]:
from sqlalchemy import select
from sqlalchemy.orm import Session

InvoiceItems = Base.classes.invoice_items
Tracks = Base.classes.tracks

session = Session(engine)

stmt = (
    select(
        InvoiceItems.InvoiceLineId,
        Tracks.Name,
        InvoiceItems.Quantity,
    )
    .join_from(InvoiceItems, Tracks, InvoiceItems.TrackId == Tracks.TrackId)
    .order_by(InvoiceItems.InvoiceLineId)
    .limit(10)
)

result = session.execute(stmt)

print("=== 10 primeiras vendas de faixas (invoice_items) ===")
for invoice_line_id, track_name, quantity in result:
    print(
        f"InvoiceLineId={invoice_line_id:3d} | "
        f"Faixa={track_name:40s} | "
        f"Quantidade={quantity}"
    )

In [ ]:
from sqlalchemy import select, func
from sqlalchemy.orm import Session

InvoiceItems = Base.classes.invoice_items
Tracks = Base.classes.tracks

session = Session(engine)

stmt = (
    select(
        Tracks.Name,
        func.sum(InvoiceItems.Quantity).label("total_vendida"),
    )
    .join_from(InvoiceItems, Tracks, InvoiceItems.TrackId == Tracks.TrackId)
    .group_by(Tracks.TrackId, Tracks.Name)
    .order_by(func.sum(InvoiceItems.Quantity).desc())
    .limit(10)
)

result = session.execute(stmt)

print("=== 10 faixas mais vendidas ===")
for track_name, total_vendida in result:
    print(f"Faixa={track_name:40s} | Total vendida={int(total_vendida)}")

In [ ]:
from sqlalchemy import select, func
from sqlalchemy.orm import Session

InvoiceItems = Base.classes.invoice_items
Tracks = Base.classes.tracks
Albums = Base.classes.albums
Artists = Base.classes.artists

session = Session(engine)

stmt = (
    select(
        Artists.Name,
        func.sum(InvoiceItems.Quantity).label("total_vendida"),
    )
    # Começamos pelas linhas de venda (invoice_items)
    .join_from(InvoiceItems, Tracks, InvoiceItems.TrackId == Tracks.TrackId)
    .join(Albums, Tracks.AlbumId == Albums.AlbumId)
    .join(Artists, Albums.ArtistId == Artists.ArtistId)
    .group_by(Artists.ArtistId, Artists.Name)
    .order_by(func.sum(InvoiceItems.Quantity).desc())
    .limit(10)
)

result = session.execute(stmt)

print("=== 10 artistas que mais venderam ===")
for artist_name, total_vendida in result:
    print(f"Artista={artist_name:30s} | Total de faixas vendidas={int(total_vendida)}")